In [ ]:
import serial
import time
import math
import csv

ser = serial.Serial(
    port='COM6',
    baudrate=115200,
    timeout=1
)

# List to store results
results = []

def send_command(command):
    ser.reset_input_buffer()  # Clear any pending input
    ser.write(f"{command}\r\n".encode())
    time.sleep(0.2)  # Wait for processing
    
    while ser.in_waiting:
        response = ser.readline().decode().strip()
        if response:  # Only print non-empty responses
            print(f"Pico response: {response}")
            # Check if response contains MPU6050 angles
            if response.startswith("MPU:"):
                return [float(x) for x in response[4:].split()]
    return None

def send_angles(pitch, yaw):
    command = f"ANGLE {pitch} {yaw}"
    return send_command(command)

def correct_pitch(target_pitch, actual_pitch, current_servo_pitch):
    # Calculate error
    error = target_pitch - actual_pitch
    
    # Apply proportional correction
    # Adjust this gain factor as needed
    Kp = 0.5
    correction = error * Kp
    
    # Calculate new servo pitch angle
    new_servo_pitch = current_servo_pitch + correction
    
    # Ensure angles stay within 0-180 range
    new_servo_pitch = max(0, min(180, new_servo_pitch))
    
    return new_servo_pitch

try:
    # Clear initial buffer
    time.sleep(2)
    ser.reset_input_buffer()
    
    # Use sample.csv file
    csv_path = "sample.csv"
    
    # Read angles from CSV
    with open(csv_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            # Get target angles from CSV
            target_pitch = float(row['pitch'])
            target_yaw = float(row['yaw'])
            
            # Ensure angles stay within 0-180 range
            target_pitch = max(0, min(180, target_pitch))
            target_yaw = max(0, min(180, target_yaw))
            
            # Initialize current servo pitch
            current_servo_pitch = target_pitch
            
            # Correction loop for pitch
            max_attempts = 3  # Maximum correction attempts
            for attempt in range(max_attempts):
                # Send angles and get actual readings
                actual_angles = send_angles(current_servo_pitch, target_yaw)
                
                if actual_angles:
                    actual_pitch, actual_yaw = actual_angles
                    pitch_error = abs(target_pitch - actual_pitch)
                    yaw_error = abs(target_yaw - actual_yaw)
                    
                    # Store results
                    results.append({
                        'Target Pitch': target_pitch,
                        'Actual Pitch': actual_pitch,
                        'Error': pitch_error
                    })
                    
                    print(f"Attempt {attempt + 1}")
                    print(f"Target angles - Pitch: {target_pitch:.2f}, Yaw: {target_yaw:.2f}")
                    print(f"Actual angles - Pitch: {actual_pitch:.2f}, Yaw: {actual_yaw:.2f}")
                    print(f"Errors - Pitch: {pitch_error:.2f}, Yaw: {yaw_error:.2f}")
                    
                    # If pitch error is acceptable, move to next position
                    if pitch_error < 2.0:  # Threshold of 2 degrees
                        break
                    
                    # Calculate corrected pitch angle
                    current_servo_pitch = correct_pitch(target_pitch, actual_pitch, current_servo_pitch)
                    print(f"Corrected servo pitch: {current_servo_pitch:.2f}")
                
                time.sleep(0.2)  # Wait between corrections
            
            # Small delay between movements
            time.sleep(0.05)

except KeyboardInterrupt:
    # Save results to CSV on keyboard interrupt
    with open('results.csv', 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['Target Pitch', 'Actual Pitch', 'Error'])
        writer.writeheader()
        writer.writerows(results)
        print("\nResults saved to results.csv")

finally:
    ser.close()
    print("Serial connection closed")